In [161]:
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Helper function (geocode)

In [162]:
with open("gkeys.key") as f:
    key = f.readline().strip()


In [163]:
def get_lat_long(addresses,key):
    """
    uses the googlev3 to parse addresses
    
    addresses: list of addresses to parse
    key: google api key
    code_country (default None): region to look into
    
    return:
    latitudes
    longitudes
    """
    import time
    from geopy import geocoders
    g = geocoders.GoogleV3(api_key=key)

    if len(addresses) > 2500:
        raise("It will take more than 24 hours to run")
#     print("Starting now")
        
    longitudes = []
    latitudes = []
    for i,address in enumerate(addresses):
        if (i>0) and (i%50 == 0): 
            print("Addressed parsed {} out of {}".format(i,len(addresses)))
            time.sleep(2)
        location = g.geocode(address,timeout=10)
        try:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
        except:
            latitudes.append(np.NaN)
            longitudes.append(np.NaN)
        print(latitudes[-1],longitudes[-1])
        
    return latitudes,longitudes

# Format data to get a uniform style

# Data from https://everytownresearch.org/school-shootings/

In [164]:
everytown = pd.read_csv("./data/everytown.csv",sep=";",header=None, parse_dates=[1])
everytown.columns = ["index","date","date_repeaded","state","city","school","type_school","gun_fired","injured","death","attempted_suicide","suicide","number_victims"]
everytown = everytown.loc[:,["date","state","school","type_school","death","suicide","number_victims"]]
everytown.to_csv("./data/everytown_formatted.tsv",index=None,sep="\t")
everytown.head(2)


,date,state,school,type_school,death,suicide,number_victims
0,2013-01-08,FL,Apostolic Revival Center Christian School,K-12,1,0,1
1,2013-01-10,CA,Taft Union High School,K-12,0,0,0


In [218]:
everytown_u = pd.read_csv("./data/everytown_updated.csv",sep=";",header=0, parse_dates=["Date"])

everytown_u.columns = ["date","city","school","state","type_school","category","attack","lat","lon"]
everytown_u["number_victims"] = np.NaN
everytown_u = everytown_u.loc[:,["date","state","school","type_school","death","suicide","number_victims","lat","lon"]]
everytown_u.to_csv("./data/everytown_u_formatted.tsv",index=None,sep="\t")
everytown_u.head(2)


/home/javiergb/Programs/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,date,state,school,type_school,death,suicide,number_victims,lat,lon
0,2018-02-14,FL,Marjory Stoneman Douglas High School,K-12,NaN,NaN,NaN,26.304503,-80.269457
1,2018-02-08,NY,The Metropolitan High School,K-12,NaN,NaN,NaN,40.827733,-73.897018


In [219]:
#For self-excitation
everytown_u_se = everytown_u[["lat","lon","number_victims","date"]]
everytown_u_se = everytown_u_se.sort_values(by="date")
everytown_u_se["number_victims"] = 1
everytown_u_se["date"] = everytown_u_se["date"].diff().dt.days
everytown_u_se["date"] = np.cumsum(everytown_u_se["date"].replace(np.NaN,0))
everytown_u_se = everytown_u_se.loc[(everytown_u_se["lon"]>-140)&(everytown_u_se["lat"]>23)]
everytown_u_se.to_csv("./data/everytown_u_self_excitation.tsv",sep=" ",index=None,header=None)
everytown_u_se.head(2)

,lat,lon,number_victims,date
289,26.640191,-81.834505,1,0.0
288,35.146303,-119.460509,1,2.0


# Data from Jim&Alyssa: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5314897/

In [198]:
#NAN value is: stewart country schools (a K-12 school)
d_type2school = {"High": "K-12", "Middle": "K-12", "college": "College", "Elementary": "K-12",np.NaN:"K-12"}

In [199]:
alyssa = pd.read_csv("./data/ShootMiami2.csv",sep=";",header=None,parse_dates=[0])
alyssa.columns = ["date","_1","_2","_3","year","type_shooing","number_victims","school","type_school","city","state","urbanity",
                 "relationship_with_attacker","gender","age","suicide","motives","population_school","lat","lon","_4","_5","_6","_7","_8"]
alyssa = alyssa.loc[:,["date","state","school","type_school","death","suicide","number_victims"]]
alyssa["death"] = (alyssa["number_victims"]>0).astype(int)
#alyssa["suicide"] = alyssa["suicide"].astype(int) 
alyssa["type_school"] = alyssa["type_school"].replace(d_type2school)
alyssa.to_csv("./data/alyssa_formatted.tsv",index=None,sep="\t")
alyssa.head(2)

/home/javiergb/Programs/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,date,state,school,type_school,death,suicide,number_victims
0,1990-08-27,NV,Eldorado High,K-12,1,0.0,1
1,1991-10-09,NY,James Monroe,K-12,1,0.0,1


In [216]:
alyssa_u = pd.read_csv("./data/ShootMiami2_updated.csv",sep=";",header=None,parse_dates=[0])
alyssa_u.columns = ["date","_1","_2","_3","year","type_shooing","number_victims","school","type_school","city","state","urbanity",
                 "relationship_with_attacker","gender","age","suicide","lat","lon"]
alyssa_u = alyssa_u.loc[:,["date","state","school","type_school","death","suicide","number_victims","lat","lon"]]
alyssa_u["death"] = (alyssa_u["number_victims"]>0).astype(int)
#alyssa["suicide"] = alyssa["suicide"].astype(int) 
alyssa_u["type_school"] = alyssa_u["type_school"].replace(d_type2school)
alyssa_u.to_csv("./data/alyssa_u_formatted.tsv",index=None,sep="\t")
alyssa_u.head(2)

/home/javiergb/Programs/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,date,state,school,type_school,death,suicide,number_victims,lat,lon
0,1990-08-27,NV,Eldorado High,K-12,1,0.0,1,38.732681,-120.811880
1,1991-10-09,NY,James Monroe,K-12,1,0.0,1,43.146905,-77.598189


In [217]:
#For self-excitation
alyssa_u_se = alyssa_u[["lat","lon","number_victims","date"]]
alyssa_u_se = alyssa_u_se.sort_values(by="date")
alyssa_u_se["date"] = alyssa_u_se["date"].diff().dt.days
alyssa_u_se["date"] = np.cumsum(alyssa_u_se["date"].replace(np.NaN,0))
alyssa_u_se = alyssa_u_se.loc[(alyssa_u_se["lon"]>-140)&(alyssa_u_se["lat"]>23)]
alyssa_u_se.to_csv("./data/alyssa_u_self_excitation.tsv",sep=" ",index=None,header=None)
alyssa_u_se.tail(2)

,lat,lon,number_victims,date
256,36.857377,-88.401604,2,10011.0
257,26.310777,-80.253225,17,10033.0


# Data from USA Today on mass shootings (http://www.gannett-cdn.com/GDContent/mass-killings/index.html)

In [202]:
usatoday = pd.read_csv("./data/USAToday.csv",sep="\t",header=None,parse_dates=[0])
usatoday.columns = ["date","city","state","type_attack","motives","number_victims"]
usatoday = usatoday.loc[usatoday["type_attack"].str.contains("hooting")]
usatoday.to_csv("./data/usatoday_formatted.tsv",index=None,sep="\t")
usatoday.head(2)

,date,city,state,type_attack,motives,number_victims
1,2006-01-30,Goleta,Calif.,Shooting,Public Killing,7
2,2006-02-21,Mesa,Ariz.,Shooting,Other,5


In [214]:
usatoday_u = pd.read_csv("./data/usatoday_u_formatted.tsv",sep="\t",header=0, parse_dates=["date"])
usatoday_u.columns = ["date","city","state","type_attack","motives","number_victims","lat","lon"]
usatoday_u.head(2)

,date,city,state,type_attack,motives,number_victims,lat,lon
0,2006-01-30,Goleta,Calif.,Shooting,Public Killing,7,34.435829,-119.827639
1,2006-02-21,Mesa,Ariz.,Shooting,Other,5,33.415184,-111.831472


In [215]:
#For self-excitation
usatoday_se = usatoday_u[["lat","lon","number_victims","date"]]
usatoday_se = usatoday_se.sort_values(by="date")
usatoday_se["date"] = usatoday_se["date"].diff().dt.days
usatoday_se["date"] = np.cumsum(usatoday_se["date"].replace(np.NaN,0))
usatoday_se = usatoday_se.loc[(usatoday_se["lon"]>-140)&(alyssa_u_se["lat"]>23)]
usatoday_se.to_csv("./data/usatoday_self_excitation.tsv",sep=" ",index=None,header=None)
usatoday_se.tail(2)

,lat,lon,number_victims,date
211,33.322662,-80.413704,4,3453.0
212,35.045630,-85.309680,5,3454.0


# Data K-12 SSDB

In [174]:
pd.options.display.max_columns=100

In [104]:
ssdb = pd.read_csv("data/K-12 SSDB (Public) - K-12 SSDB (Public) Linked.tsv",sep="\t",engine="python",skiprows=1,quoting=3,
                   parse_dates=["Date"],infer_datetime_format=True)
ssdb["Address"] = ssdb["School"] + ", " + ssdb["City"] + ", " + ssdb["State"] + ", United States"

ssdb = ssdb.loc[:,["Date","State","School","Type_school","Death",
            "Suicide (or attempted suicide) by Shooter (Y/N)",
            "Killed (includes shooter)","Reliability Score (1-5)","Category","Address"]]

ssdb.columns = ["date","state","school","type_school","death","suicide","number_victims","reliability","category","address"]
ssdb["death"] = (ssdb["number_victims"] > 0).astype(int)
ssdb["type_school"] = "K-12"
ssdb["suicide"] = ssdb["suicide"].map({"Y":1,"N":0})
ssdb.head()

/home/javiergb/Programs/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,date,state,school,type_school,death,suicide,number_victims,reliability,category,address
0,1970-01-05,DC,Hine Junior High School,K-12,1,0.0,1,3.0,Accidental,"Hine Junior High School, Washington, DC, Unite..."
1,1970-01-05,DC,Sousa Junior High,K-12,0,0.0,0,3.0,Accidental,"Sousa Junior High, Washington, DC, United States"
2,1970-01-05,DC,Unnamed High School,K-12,0,0.0,0,2.0,Robbery,"Unnamed High School, Washington, DC, United St..."
3,1970-02-06,OH,John F. Kennedy High School,K-12,0,0.0,0,2.0,Escalation of Dispute,"John F. Kennedy High School, Cleveland, OH, Un..."
4,1970-03-23,CA,David Starr Jordan High School,K-12,0,0.0,0,2.0,Unknown,"David Starr Jordan High School, Long Beach, CA..."


In [89]:
results = get_lat_long(ssdb["address"],key)

Starting now
38.8923472 -76.97896759999999
38.8839745 -76.9531487
38.9672873 -77.0193303
41.4494718 -81.5607
33.8722207 -118.1840179
34.2161319 -92.05407439999999
39.76428 -75.53298
26.4605387 -80.1128433
41.2493595 -111.9511011
31.7337929 -106.3723188
25.8051806 -80.2267373
36.1339392 -86.76598419999999
41.5312612 -90.57645269999999
42.6389216 -83.29104679999999
36.1887559 -95.9708218
36.1631124 -86.8124993
40.7626493 -82.5429017
41.7182472 -87.62071139999999
44.2487795 -88.4333529
40.0714999 -75.1743432
40.0312226 -74.8821102
32.6996702 -96.8157572
41.5352632 -81.6104739
32.228453 -110.963284
40.2931721 -76.89852239999999
42.1024415 -86.457231
42.7686922 -78.618675
40.139574 -79.869912
33.5705848 -101.8155127
42.937193 -78.8791182
44.9489021 -93.1484443
37.1873005 -77.3950615
39.3256435 -76.5969191
39.3147394 -76.6549423
39.3325577 -76.6894252
32.4291476 -104.234687
36.5873018 -94.7821621
39.2917249 -76.687831
34.0260066 -118.1933118
39.3044007 -76.6526347
38.9345928 -77.021391299999

30.27762299999999 -81.61151360000001
40.678748 -73.87521319999999
40.5827332 -73.9636603
29.806465 -95.695424
39.9282102 -75.3385522
28.100798 -82.5035101
41.6337012 -80.12619649999999
32.5341636 -92.7865659
30.3224789 -97.6898391
33.1994604 -87.5079894
27.8807696 -82.7238871
41.68770840000001 -71.2692521
42.35426959999999 -83.2478407
34.1769133 -117.3094582
30.2329676 -97.74418539999999
41.81411749999999 -87.60377869999999
26.6197477 -81.86507189999999
42.1143602 -87.7457302
40.8439944 -73.88452099999999
32.2803997 -90.2417644
42.401428 -83.136196
21.3849016 -157.9300814
43.05099020000001 -87.9303396
32.3175301 -90.2025396
42.38358909999999 -83.11178269999999
34.496686 -82.6645757
32.756467 -96.751881
Addressed parsed 350 out of 1377
35.2409912 -80.7819896
33.4321554 -82.0740562
41.8648712 -87.66659659999999
32.7872032 -80.03834859999999
28.5723712 -81.88571220000001
30.4763011 -91.1517733
30.2549618 -81.7375978
39.2917249 -76.687831
30.6966263 -88.0532326
39.281707 -76.63856190000001

32.249322 -107.7803914
35.7936675 -95.23925539999999
61.2320185 -149.7334299
35.096572 -106.63567
28.2712236 -82.68880659999999
35.6193959 -82.6263124
Addressed parsed 650 out of 1377
37.2871585 -120.4951994
41.2136804 -95.9463689
39.93732240000001 -75.2503682
41.8853859 -87.76621870000001
43.0803044 -83.7055136
32.0547935 -81.11589099999999
40.7737774 -80.7598931
34.0148897 -95.51072239999999
32.3104298 -110.9937843
39.647822 -104.922373
31.562682 -110.2992417
26.632908 -80.07081
47.4924296 -122.2324887
40.7993967 -81.3787701
38.9236892 -82.28931740000002
39.2650112 -84.56178899999999
29.9396352 -90.09104839999999
44.9941737 -93.2997876
33.6192444 -112.1958595
32.9790746 -96.8922292
34.260973 -118.5252465
32.825762 -117.104945
37.9520704 -122.346888
25.9520128 -80.2171563
37.7215796 -122.4407711
34.1605528 -119.182211
39.3163674 -76.5896443
42.434046 -83.0027346
32.7554975 -117.0979213
32.857439 -116.9691749
38.8862713 -76.82034879999999
41.2492558 -76.9964607
47.370635 -122.120278
32

41.8404753 -87.6181897
41.8608315 -87.6274658
43.0984267 -83.71533889999999
41.8643651 -87.7011464
42.137546 -80.0594297
38.5648931 -90.1643363
35.650075 -78.46562
34.0661729 -118.0158634
35.8024413 -78.4852234
42.38358909999999 -83.11178269999999
37.6565801 -122.0834135
41.0856093 -73.43315319999999
43.5167054 -96.7217857
35.0843529 -78.9917528
30.390581 -81.7077094
29.892453 -95.6046483
33.9353785 -118.268409
43.7762373 -87.7194166
42.909229 -77.2755766
29.562631 -90.3650057
37.7753816 -122.4216371
42.57303539999999 -92.77852999999999
35.0843529 -78.9917528
41.0593697 -73.5364836
37.6618987 -77.4818006
40.99595619999999 -72.5283285
33.7798928 -78.97076849999999
33.7768888 -118.140621
39.955223 -75.2353265
32.5040265 -93.774143
32.6111637 -88.184477
Addressed parsed 1000 out of 1377
34.7408573 -86.7414711
36.0101194 -83.9591262
39.5867127 -105.1019619
47.268669 -122.459469
36.8141251 -76.3798262
33.9560708 -118.2093189
36.5297077 -82.38709449999999
42.4213221 -83.1646032
33.6848892 -7

46.8471045 -114.0095543
37.2188117 -93.3197647
35.0236537 -90.04978729999999
38.8435107 -94.6378824
41.41493579999999 -81.5401014
38.4377203 -82.0057994
46.5790038 -120.6089187
39.4221685 -76.2860865
26.652448 -80.179552
Addressed parsed 1300 out of 1377
30.3822126 -81.7092892
38.7876032 -89.9688781
39.7626269 -104.9691745
39.1781352 -75.5217558
35.2367612 -80.8126726
42.9257517 -85.6273537
41.65077230000001 -90.5891055
40.8691591 -73.90187569999999
41.816389 -71.42206039999999
36.9950502 -121.5763444
41.7300091 -87.6069524
35.011956 -90.051048
36.233333 -115.136111
26.5417603 -80.08350980000002
47.887831 -122.2363741
34.723494 -86.56701559999999
34.1951647 -118.4482843
35.2261011 -80.7440106
40.0374048 -75.15031689999999
39.2368578 -76.60697259999999
33.0358379 -96.85837699999999
34.6491779 -92.36568129999999
61.2132436 -149.880805
45.53776449999999 -122.5674
35.6012279 -89.24746209999999
36.8705817 -76.1981392
36.1857094 -86.6779701
42.9894476 -71.4796472
42.3612493 -83.220766
35.120

In [90]:
ssdb["lat"],ssdb["lon"] = results

In [91]:
ssdb.to_csv("./data/ssdb_u_formatted.tsv",sep="\t")

In [212]:
ssdb = pd.read_csv("./data/ssdb_u_formatted.tsv",sep="\t",index_col=0,parse_dates=["date"])

In [213]:
#For self-excitation
ssdb_se = ssdb[["lat","lon","number_victims","date"]]
ssdb_se = ssdb_se.sort_values(by="date")
ssdb_se["date"] = ssdb_se["date"].diff().dt.days
ssdb_se["date"] = np.cumsum(ssdb_se["date"].replace(np.NaN,0))
ssdb_se = ssdb_se.loc[(ssdb_se["lon"]>-140)&(alyssa_u_se["lat"]>23)]
ssdb_se.to_csv("./data/ssdb_self_excitation.tsv",sep=" ",index=None,header=None)
ssdb_se.tail(2)

,lat,lon,number_victims,date
257,32.504027,-93.774143,0,5352.0
235,33.407736,-91.038557,1,8667.0


# Data Pah

In [207]:
pah = pd.read_csv("data/pah_us_school_gun_violence.csv",index_col=0,parse_dates=["Date"])
pah.columns = ["date","city","state","area_type","type_school","number_victims"]
pah["type_school"] = pah["type_school"].map({'HS':"K-12", 'MS':"K-12", 'C':"College", 'ES':"K-12"})
pah["Address"] = pah["city"] + ", " + pah["state"] + ", United States"
pah.tail()

,date,city,state,area_type,type_school,number_victims,Address
184,2013-11-03,Lithonia,Georgia,rural,K-12,0,"Lithonia, Georgia, United States"
84,2013-11-13,Pittsburgh,Pennsylvania,urban,K-12,0,"Pittsburgh, Pennsylvania, United States"
86,2013-12-04,Winter Garden,Florida,suburban,K-12,0,"Winter Garden, Florida, United States"
351,2013-12-13,Centennial,Colorado,suburban,K-12,2,"Centennial, Colorado, United States"
121,2013-12-19,Fresno,California,urban,K-12,0,"Fresno, California, United States"


In [ ]:
results2 = get_lat_long(pah["Address"],key)

In [ ]:
pah["lat"],pah["lon"] = results2

In [ ]:
#update data Pah with Alyssa + what manual update
pah = pd.concat([pah,alyssa_u.loc[alyssa_u["date"]>pah["date"].max()]])

In [139]:
#only attacks with victims
pah = pah.loc[pah["number_victims"]>0]

In [140]:
pah.to_csv("./data/pah_u_formatted.tsv",sep="\t")

In [210]:
pah = pd.read_csv("./data/pah_u_formatted.tsv",sep="\t",index_col=0,parse_dates=["date"])
pah.head()

,Address,Unnamed: 0.1,area_type,city,date,death,lat,lon,number_victims,school,state,suicide,type_school
0,"Centerville, Tennessee, United States",208.0,suburban,Centerville,1990-05-20,NaN,35.778957,-87.466959,1,NaN,Tennessee,NaN,K-12
1,"Las Vegas, Nevada, United States",30.0,urban,Las Vegas,1990-08-26,NaN,36.169941,-115.139830,1,NaN,Nevada,NaN,K-12
3,"Compton, California, United States",186.0,urban,Compton,1991-04-23,NaN,33.895849,-118.220071,1,NaN,California,NaN,K-12
4,"Crosby, Texas, United States",31.0,rural,Crosby,1991-09-18,NaN,29.911888,-95.062151,1,NaN,Texas,NaN,K-12
5,"Bronx, New York, United States",154.0,urban,Bronx,1991-10-08,NaN,40.844782,-73.864827,1,NaN,New York,NaN,K-12


In [211]:
#For self-excitation
pah_se = pah[["lat","lon","number_victims","date"]]

pah_se = pah_se.sort_values(by="date")
pah_se["date"] = pah_se["date"].diff().dt.days
pah_se["date"] = np.cumsum(pah_se["date"].replace(np.NaN,0))
pah_se = pah_se.loc[(pah_se["lon"]>-140)&(pah_se["lat"]>23)]
pah_se.to_csv("./data/pah_self_excitation.tsv",sep=" ",index=None,header=None)
pah_se.tail(2)

,lat,lon,number_victims,date
256,36.857377,-88.401604,2,10110.0
257,26.310777,-80.253225,17,10132.0


# Format tweet count

In [152]:
!cat "./data/datav2/14.08.14.tgzSS.txt"

cat: ./data/datav2/14.08.14.tgzSS.txt: No such file or directory


In [ ]:
!ls

In [121]:

def format_tweets():
    import os
    import pylab as plt
    import numpy as np
    from datetime import datetime
    import pandas as pd

    listFiles = os.listdir("./data/datav2/")
    with open("./data/all_tweets_formatted.tsv","w+") as fout:
        fout.write("date\tall\tschool_shooting\tmass_shooting\tmass_murder\tshooting\n")
    for fileT in listFiles:
        if ".tgz" in fileT:
            name = fileT[:-10]
            name = "20" + name[-2:] +"-"+ name[3:5] + "-" + name[0:2]
        elif ".gz" in fileT:
            name = fileT[:-15]
        else: 
            print(fileT)
            continue
            
        date = datetime(year=int(name[0:4]),month=int(name[5:7]),day=int(name[-2:]))

        
        with open("./data/datav2/"+fileT) as f, open("./data/all_tweets_formatted.tsv","a+") as fout:
            lines = [float(_) for _ in f.readlines()]
            fout.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(name,*lines))
    


In [155]:
df = pd.read_csv("./data/all_tweets_formatted.tsv",sep="\t",parse_dates=["date"])
#Sum the last files (one file per 15 min or so)
df = df.groupby("date").sum().reset_index()
df.to_csv("./data/all_tweets_formatted.tsv",sep="\t",index=None)

In [122]:
format_tweets()

tempFiles.txt


,date,city,state,area_type,type_school,number_victims,Address
184,2013-11-03,Lithonia,Georgia,rural,K-12,0,"Lithonia, Georgia, United States"
84,2013-11-13,Pittsburgh,Pennsylvania,urban,K-12,0,"Pittsburgh, Pennsylvania, United States"
86,2013-12-04,Winter Garden,Florida,suburban,K-12,0,"Winter Garden, Florida, United States"
351,2013-12-13,Centennial,Colorado,suburban,K-12,2,"Centennial, Colorado, United States"
121,2013-12-19,Fresno,California,urban,K-12,0,"Fresno, California, United States"
